In [1]:
# Import required libraries
from pgmpy.models import DiscreteBayesianNetwork 
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

c:\Users\pc\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1: Define the Bayesian Network structure
# LoanApproval depends on Income and CreditScore
model = DiscreteBayesianNetwork([
    ('Income', 'LoanApproval'),
    ('CreditScore', 'LoanApproval')
])

In [3]:
# Step 2: Define Conditional Probability Distributions (CPDs)

# Prior probability of Income (High=0.6, Low=0.4)
cpd_income = TabularCPD(
    variable='Income',
    variable_card=2,
    values=[[0.6], [0.4]],
    state_names={'Income': ['High', 'Low']}
)

In [4]:
# Prior probability of CreditScore (Good=0.7, Bad=0.3)
cpd_credit = TabularCPD(
    variable='CreditScore',
    variable_card=2,
    values=[[0.7], [0.3]],
    state_names={'CreditScore': ['Good', 'Bad']}
)

In [5]:
# Conditional probability of LoanApproval given Income & CreditScore
# Rows = LoanApproval states (Yes, No)
# Columns = combinations of (Income, CreditScore)
cpd_loan = TabularCPD(
    variable='LoanApproval',
    variable_card=2,
    values=[
        [0.95, 0.70, 0.60, 0.20],  # Yes
        [0.05, 0.30, 0.40, 0.80]   # No
    ],
    evidence=['Income', 'CreditScore'],
    evidence_card=[2, 2],
    state_names={
        'LoanApproval': ['Yes', 'No'],
        'Income': ['High', 'Low'],
        'CreditScore': ['Good', 'Bad']
    }
)

In [6]:
# Step 3: Add CPDs to the model
model.add_cpds(cpd_income, cpd_credit, cpd_loan)

In [7]:
# Step 4: Create an inference object
inference = VariableElimination(model)

In [8]:
# Step 5: Query -> P(LoanApproval | Income=High, CreditScore=Bad)
result = inference.query(
    variables=['LoanApproval'],
    evidence={'Income': 'High', 'CreditScore': 'Bad'}
)

In [10]:
# Step 6: Print result
print("Probability of Loan Approval given High Income and Bad Credit:")
print(result)

Probability of Loan Approval given High Income and Bad Credit:
+-------------------+---------------------+
| LoanApproval      |   phi(LoanApproval) |
+===================+=====================+
| LoanApproval(Yes) |              0.7000 |
+-------------------+---------------------+
| LoanApproval(No)  |              0.3000 |
+-------------------+---------------------+
